In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import scipy.stats as stats
from scipy.stats import pearsonr
from tqdm import tqdm
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
df = pd.read_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/dataframe_compressed.parquet')

In [3]:
df_model = pd.read_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/df_cleaned_for_model.parquet')

In [32]:
df_old = pd.read_csv('C://Users/Isi/anaconda3/envs/FireBrigade/Data/df_clean_before_NaNs_with_datetime_infos_with_geo.csv', index_col = 'IncidentNumber', dtype={'IncidentNumber': 'str'}, parse_dates=['DateTimeCall'])

In [36]:
df_cyc = df_old[['Hour', 'Month', 'WeekDay']]

In [61]:
df_model

,IncidentGroup_Fire,IncidentGroup_Special Service,AggregatedPropertyCategory_Outdoor,AggregatedPropertyCategory_Residential,AggregatedPropertyCategory_Vehicle,CellEastingNorthing2500_502500-175000,CellEastingNorthing2500_502500-177500,CellEastingNorthing2500_502500-180000,CellEastingNorthing2500_502500-182500,CellEastingNorthing2500_502500-187500,...,IsBankholiday,IsWeekend,DistanceStationLog,TotalResponseTimeLog,Hour_sin,Hour_cos,Weekday_sin,Weekday_cos,Month_sin,Month_cos
IncidentNumber,,,,,,,,,,,,,,,,,,,,,
235138081,0,1,0,0,1,0,0,0,0,0,...,1,0,7.207600,5.765191,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
2091,1,0,1,0,0,0,0,0,0,0,...,1,0,6.454777,5.730100,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
3091,1,0,1,0,0,0,0,0,0,0,...,1,0,5.940170,5.347108,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
5091,1,0,1,0,0,0,0,0,0,0,...,1,0,7.031189,5.451038,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
6091,0,0,0,1,0,0,0,0,0,0,...,1,0,6.281069,5.147494,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
067897-30042024,0,1,0,1,0,0,0,0,0,0,...,0,0,6.748761,5.572154,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000
067898-30042024,0,0,0,1,0,0,0,0,0,0,...,0,0,7.248166,5.666427,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000
067896-30042024,0,0,0,1,0,0,0,0,0,0,...,0,0,7.576672,6.190315,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000


In [42]:
#Converting Hours to cyclic
p = 24
df_cyc.loc[:,'Hour_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'Hour'] / p)
df_cyc.loc[:,'hour_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'Hour'] / p)

In [44]:
#Converting Weekdays to cyclic
p = 7
df_cyc.loc[:,'Weekday_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'WeekDay'] / p)
df_cyc.loc[:,'Weekday_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'WeekDay'] / p)

C:\Users\Isi\AppData\Local\Temp\ipykernel_9768\313601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cyc.loc[:,'Weekday_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'WeekDay'] / p)
C:\Users\Isi\AppData\Local\Temp\ipykernel_9768\313601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cyc.loc[:,'Weekday_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'WeekDay'] / p)


In [46]:
#Converting Months to cyclic
p = 12
df_cyc.loc[:,'Month_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'Month'] / p)
df_cyc.loc[:,'Month_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'Month'] / p)

C:\Users\Isi\AppData\Local\Temp\ipykernel_9768\3962146605.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cyc.loc[:,'Month_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'Month'] / p)
C:\Users\Isi\AppData\Local\Temp\ipykernel_9768\3962146605.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cyc.loc[:,'Month_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'Month'] / p)


In [18]:
df_model['DistanceStationLog'] = np.log(df_model['DistanceStation'])

In [20]:
df_model['TotalResponseTimeLog'] = np.log(df_model['TotalResponseTime'])

In [22]:
df_model = df_model.drop(['TotalResponseTime', 'DistanceStation'], axis = 1)

In [62]:
X = df_model.drop(['TotalResponseTimeLog'], axis = 1)
y = df_model['TotalResponseTimeLog']

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [64]:
# saving the datasets to files
df_model.to_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/df_cleaned_for_model.parquet')
X_train.to_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/X_train.parquet')
X_test.to_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/X_test.parquet')

In [91]:
names = ['AggregatedPropertyCategory_Outdoor', 'AggregatedPropertyCategory_Vehicle', 'DistanceStation', 'IncidentGroup_Special Service', 'StopCodeDescription_Secondary Fire', 'StopCodeDescription_Special Service']

In [80]:
X

,IncidentGroup_Fire,IncidentGroup_Special Service,AggregatedPropertyCategory_Outdoor,AggregatedPropertyCategory_Residential,AggregatedPropertyCategory_Vehicle,CellEastingNorthing2500_502500-175000,CellEastingNorthing2500_502500-177500,CellEastingNorthing2500_502500-180000,CellEastingNorthing2500_502500-182500,CellEastingNorthing2500_502500-187500,...,CellEastingNorthing2500_560000-187500,IsBankholiday,IsWeekend,DistanceStationLog,Hour_sin,Hour_cos,Weekday_sin,Weekday_cos,Month_sin,Month_cos
IncidentNumber,,,,,,,,,,,,,,,,,,,,,
235138081,0,1,0,0,1,0,0,0,0,0,...,0,1,0,7.207600,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
2091,1,0,1,0,0,0,0,0,0,0,...,0,1,0,6.454777,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
3091,1,0,1,0,0,0,0,0,0,0,...,0,1,0,5.940170,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
5091,1,0,1,0,0,0,0,0,0,0,...,0,1,0,7.031189,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
6091,0,0,0,1,0,0,0,0,0,0,...,0,1,0,6.281069,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
067897-30042024,0,1,0,1,0,0,0,0,0,0,...,0,0,0,6.748761,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000
067898-30042024,0,0,0,1,0,0,0,0,0,0,...,0,0,0,7.248166,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000
067896-30042024,0,0,0,1,0,0,0,0,0,0,...,0,0,0,7.576672,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000


In [78]:
model = LinearRegression()

In [79]:
model.fit(X_train, y_train)

LinearRegression()

In [81]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [82]:
print(f"Mean squared error: {mse}")
print(f"R2 score: {r2}")
print(f"Mean absolute error: {mean_absolute_error(y_test, y_pred)}")
print(f"Root Mean squared error: {np.sqrt(mse)}")
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5)
print("Cross-validated MSE:", -scores.mean())

Mean squared error: 4.2221606120952416e+17
R2 score: -2.6905247175520824e+18
Mean absolute error: 1210401.980178317
Root Mean squared error: 649781548.8373952
Cross-validated MSE: 9.458950028671115e+18


In [75]:
#print("Model Coefficients:", model.coef_)
print("Model Intercept:", model.intercept_)

Model Intercept: 3.9949890287834595


In [76]:
pd.crosstab(y_test, y_pred)

col_0,4.314085e+00,4.314958e+00,4.315283e+00,4.318432e+00,4.320841e+00,4.321609e+00,4.323414e+00,4.324804e+00,4.325713e+00,4.332466e+00,...,6.645476e+00,6.654803e+00,6.665300e+00,6.670273e+00,6.670535e+00,6.714258e+00,6.741308e+00,6.763540e+00,1.210626e+10,3.601419e+11
TotalResponseTimeLog,,,,,,,,,,,,,,,,,,,,,
4.094345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.110874,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.127134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.143135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.158883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7.085901,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7.086738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7.087574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
